In [1]:
# import statements
import pandas as pd
import pandasql as ps

In [2]:
# create dataframe from json file
raw_data_df = pd.read_json(r'data_2015_05.json', orient='records', lines=True)

In [3]:
# show top 10 rows
raw_data_df.head(10)

,body,score_hidden,archived,name,author,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,author_flair_text,author_flair_css_class,distinguished
0,That wasn't just a personal belief of his. It'...,False,False,t1_cr7h0ln,tejon,0,1431494075,t5_2cneq,t3_35ql9d,t1_cr7fnj4,136,1433034878,0,0,cr7h0ln,politics,136,NaN,NaN,NaN
1,I think you're on to something there. But to b...,False,False,t1_crpxkmu,Tangpo,0,1432999141,t5_2cneq,t3_37tyhm,t1_crpw8y8,17,1433393342,0,0,crpxkmu,politics,17,NaN,NaN,NaN
2,They've also been rather clear on defining a f...,False,False,t1_cr43h2c,Qu1nlan,0,1431220695,t5_2cneq,t3_35eepj,t1_cr439ea,19,1432870611,0,0,cr43h2c,politics,19,NaN,NaN,NaN
3,God bless you.,False,False,t1_cr5rn0g,broncoz33,0,1431370502,t5_2qh03,t3_35liga,t1_cr5qa14,13,1432899458,0,0,cr5rn0g,gaming,13,NaN,NaN,NaN
4,Western corporations don't give a rats arse to...,False,False,t1_cra1d35,critfist,0,1431701414,t5_2qh13,t3_361cco,t1_cra0qd0,-4,1433079241,0,0,cra1d35,worldnews,-4,NaN,NaN,NaN
5,[deleted],False,False,t1_cqvze0l,[deleted],0,1430571181,t5_2qh13,t3_34lnbr,t1_cqvye5y,-3,1432729493,0,0,cqvze0l,worldnews,-3,NaN,NaN,NaN
6,http://en.wikipedia.org/wiki/Celebration_Cruis...,False,False,t1_crqsdm6,Atheist101,0,1433078136,t5_323oy,t3_37v2sr,t1_crqdes4,-4,1433496812,0,0,crqsdm6,CasualConversation,-4,NaN,NaN,NaN
7,"I'm not attacking them, I'm making an observat...",False,False,t1_crahjf8,utb040713,0,1431728711,t5_mouw,t3_3622fk,t1_cragizb,0,1433087005,0,0,crahjf8,science,0,NaN,NaN,NaN
8,"Well, that's a vile and disgusting thing to sa...",False,False,t1_cr4qp74,Fractal_Soul,0,1431286132,t5_2cneq,t3_35ffem,t1_cr4div8,0,1432881767,0,0,cr4qp74,politics,0,NaN,NaN,NaN
9,Are you 15? Lol,False,False,t1_crcbtar,chuchubaba,0,1431900496,t5_2qh03,t3_369dwg,t1_crc88ht,0,1433118815,0,0,crcbtar,gaming,0,NaN,NaN,NaN


In [4]:
# filter raw data
sql_query = " select body" \
            "       , name" \
            "       , link_id" \
            "       , parent_id" \
            "       , score" \
            " from raw_data_df" \
            " where length(body) < 200 and body <> '[deleted]'"
sdf = ps.sqldf(sql_query) # csak sdf lehet a neve

In [5]:
del raw_data_df

In [6]:
response_df = sdf.rename(columns={'body': 'response'})

In [14]:
response_df.head(10)

,response,name,link_id,parent_id,score
0,They've also been rather clear on defining a f...,t1_cr43h2c,t3_35eepj,t1_cr439ea,19
1,God bless you.,t1_cr5rn0g,t3_35liga,t1_cr5qa14,13
2,Western corporations don't give a rats arse to...,t1_cra1d35,t3_361cco,t1_cra0qd0,-4
3,http://en.wikipedia.org/wiki/Celebration_Cruis...,t1_crqsdm6,t3_37v2sr,t1_crqdes4,-4
4,"Well, that's a vile and disgusting thing to sa...",t1_cr4qp74,t3_35ffem,t1_cr4div8,0
5,Are you 15? Lol,t1_crcbtar,t3_369dwg,t1_crc88ht,0
6,"Oh okay, cause a lot of people post similar ""t...",t1_crjr1yc,t3_372bqp,t1_crj8tdk,0
7,&gt; according to analysts who monitor web cha...,t1_cr2nszt,t3_357t9u,t3_357t9u,0
8,"Minavo Communications provides IVR solutions, ...",t1_cr5cntq,t3_35kx45,t3_35kx45,1
9,"Pfft, Convolvulaceae is where it's at. ;-)",t1_crmt7hn,t3_37gbrz,t1_crmsccb,1


In [8]:
# create query-response pairs
sql_query = " select inp.body" \
            "       , resp.response" \
            "       , resp.score" \
            " from response_df resp" \
            " left join sdf inp" \
            " on resp.parent_id = inp.name" \
            " where inp.body is not null"
sdf = ps.sqldf(sql_query)

In [9]:

xy_df = sdf.rename(columns={'body': 'input'})

In [10]:
del sdf

In [15]:
# show top 10 rows
xy_df.head(10)

,input,response,score
0,They've also been rather clear on defining a f...,"No, they have not. What gave you that idea? Th...",-23
1,Western corporations don't give a rats arse to...,"wow, ok have you ever heard of a thing called ...",6
2,http://en.wikipedia.org/wiki/Celebration_Cruis...,"That's not the cruise line from the ""radio sta...",11
3,http://en.wikipedia.org/wiki/Celebration_Cruis...,http://i.imgur.com/oeLtsDA.jpg,2
4,What was the conversation? I just killed them.,"""Something something something, then I raped t...",1
5,Well it just shows us past behavior and what o...,"Guess you aren't a big believer in ""firsts""?\n",1
6,Solid advice to avoid a shaky end for /u/TheEa...,Agreed!,1
7,The whole schools discrimination provision was...,The fact my daughter is bottom of the list in ...,1
8,Hahaha yeah we can do that too. I just thought...,Nothing wrong with multitasking during a movie.,1
9,"Thank you!\n\nYeah, I like them and find them ...",Understandable. It's not a real exam if you do...,2


In [12]:
# export to json
# 960411 sor, 5 oszlop
response_df.to_json('filtered_data_2015_05.json', orient='records', lines=True)
# 485400 sor, 3 oszlop
xy_df.to_json('xy_data_2015_05.json', orient='records', lines=True)

In [16]:
del response_df
del xy_df